<h1>SQL</h1>

基本語法
http://www.1keydata.com/tw/sql/sqlselect.html

In [2]:
import math, random, re
from collections import defaultdict

In [3]:
class Table:
    def __init__(self, columns):  #初始設定欄位名稱    #self 可使用.
        self.columns = columns   #放入ROW的資料會變成COLUMN名稱
        self.rows = []           #ROW只能放入list

    def __repr__(self):
        """pretty representation of the table: columns then rows"""
        return str(self.columns)+ "\n" + "\n" .join(map(str, self.rows))    #把每個ROW  用COLUM做分割
    #.join用法示例 
    #li = ['my','name','is','bob'] 
    #' '.join(li) 
    #'my name is bob'
    
    

    def insert(self, row_values):
        if len(row_values) != len(self.columns):
            raise TypeError("wrong number of elements")   #插入陣列的長度 若不等於原先長度  會回傳錯誤
        row_dict = dict(zip(self.columns, row_values))  #dict會返回{}形式  (轉換成字典)
        self.rows.append(row_dict)   #將新值 回傳至初始之空欄位
        
         #################################################################################       

    def update(self, updates, predicate):
           for row in self.rows:
            if predicate(row):    #创建一个迭代器，只要函数predicate(item)为True，就將更新的元素放入陣列
                                    
                for column, new_value in updates.items():
                    row[column] = new_value

    def delete(self, predicate=lambda row: True):
        """delete all rows matching predicate
        or all rows if no predicate supplied"""
        self.rows = [row for row in self.rows if not(predicate(row))]
        
        #####################################################################################

    def select(self, keep_columns=None, additional_columns=None):            #keep_colum為選擇欄位   
                                                                            #adtion_colums為函數條件  類似having

        if keep_columns is None:         # if no columns specified,
            keep_columns = self.columns  # return all columns

        if additional_columns is None:
            additional_columns = {}

        # new table for results
        result_table = Table(keep_columns + list(additional_columns.keys()))       #使用.KEY()  能抓取所有鍵 (前面那欄)
                                                                                   #使用.value()  回傳所有值
            
        for row in self.rows:
            new_row = [row[column] for column in keep_columns]    
            for column_name, calculation in additional_columns.items():    #將附加條件的 鑑名   和值  放入table回傳
                new_row.append(calculation(row))
            result_table.insert(new_row)

        return result_table
    
    

    def where(self, predicate=lambda row: True):     #針對欄位下條件
        """return only the rows that satisfy the supplied predicate"""    #給定lambda會自動蝶帶字典中符合條件的鑑和值
        where_table = Table(self.columns)
        where_table.rows = list(filter(predicate, self.rows))               #def is_odd(n):
                                                                            #return n % 2 == 1
                                                                            #filter(is_odd, [1, 2, 4, 5, 6, 9, 10, 15])
                                                                            # 结果: [1, 5, 9, 15]
        return where_table

    def limit(self, num_rows=None):
        """return only the first num_rows rows"""
        limit_table = Table(self.columns)
        limit_table.rows = (self.rows[:num_rows]
                            if num_rows is not None
                            else self.rows)
        return limit_table

    
    
    ################################################################
    
    
    
    def group_by(self, group_by_columns, aggregates, having=None):      #having:加條件(針對函數)   #此處aggreates 是函數  having是函數條件

        grouped_rows = defaultdict(list)     #創造一個字典  裡面都放list  list可將元素拆開  #>>> a_tuple = ('ready', 'fire', 'aim')
                                                                                        #>>> list(a_tuple)
                                                                                        #['ready', 'fire', 'aim']

        # populate groups
        for row in self.rows:
            key = tuple(row[column] for column in group_by_columns)  #將元素  包成位元組
                                                                        #此處轉換為tuple因為函數內的參數是以tuple傳遞
            grouped_rows[key].append(row)   #append  加元素到尾部

        result_table = Table(group_by_columns + list(aggregates.keys()))

        for key, rows in grouped_rows.items():
            if having is None or having(rows):   #此處之having為  函數中加入之搜尋條件
                new_row = list(key)      #為了放入insert將key轉換成list
                for aggregate_name, aggregate_fn in aggregates.items():
                    new_row.append(aggregate_fn(rows))   #將附加條件 加入尾部
                result_table.insert(new_row)

        return result_table

    def order_by(self, order):
        new_table = self.select()       # make a copy
        new_table.rows.sort(key=order)
        return new_table

    def join(self, other_table, left_join=False):

        join_on_columns = [c for c in self.columns           # columns in
                           if c in other_table.columns]      # both tables      #兩個表格都有的欄

        additional_columns = [c for c in other_table.columns # columns only
                              if c not in join_on_columns]   # in right table  #新欄位

        # all columns from left table + additional_columns from right table
        join_table = Table(self.columns + additional_columns)

        for row in self.rows:
            def is_join(other_row):
                return all(other_row[c] == row[c] for c in join_on_columns)

            other_rows = other_table.where(is_join).rows           #找出 兩邊都有的列

            # each other row that matches this one produces a result row
            for other_row in other_rows:
                join_table.insert([row[c] for c in self.columns] +              #將原始的列  加入欲插入的列
                                  [other_row[c] for c in additional_columns])

            # if no rows match and it's a left join, output with Nones
            if left_join and not other_rows:
                join_table.insert([row[c] for c in self.columns] +           #新增欄位有空值的話   補上NONE
                                  [None for c in additional_columns])

        return join_table

In [4]:
if __name__ == "__main__":

    users = Table(["user_id", "name", "num_friends"])
    print(users)

['user_id', 'name', 'num_friends']



In [5]:
if __name__ == "__main__":

    users = Table(["user_id", "name", "num_friends"])
    users.insert([0, "Hero", 0])
    users.insert([1, "Dunn", 2])
    users.insert([2, "Sue", 3])
    users.insert([3, "Chi", 3])
    users.insert([4, "Thor", 3])
    users.insert([5, "Clive", 2])
    users.insert([6, "Hicks", 3])
    users.insert([7, "Devin", 2])
    users.insert([8, "Kate", 2])
    users.insert([9, "Klein", 3])
    users.insert([10, "Jen", 1])

    print("users table")
    print(users)
    print()

users table
['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}



# SELECT

# SELECT * FROM users

In [8]:
print("users.select()")
print(users.select())
print()

 #回傳的第一行   即為函數中的list值

users.select()
['user_id', 'name', 'num_friends']
{'user_id': 0, 'num_friends': 0, 'name': 'Hero'}
{'user_id': 1, 'num_friends': 2, 'name': 'Dunn'}
{'user_id': 2, 'num_friends': 3, 'name': 'Sue'}
{'user_id': 3, 'num_friends': 3, 'name': 'Chi'}
{'user_id': 4, 'num_friends': 3, 'name': 'Thor'}
{'user_id': 5, 'num_friends': 2, 'name': 'Clive'}
{'user_id': 6, 'num_friends': 3, 'name': 'Hicks'}
{'user_id': 7, 'num_friends': 2, 'name': 'Devin'}
{'user_id': 8, 'num_friends': 2, 'name': 'Kate'}
{'user_id': 9, 'num_friends': 3, 'name': 'Klein'}
{'user_id': 10, 'num_friends': 1, 'name': 'Jen'}



# SELECT * FROM users LIMIT 2

In [7]:
print("users.limit(3)")
print(users.limit(3))
print()

users.limit(3)
['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}



# SELECT user_id FROM users

In [10]:
print("users.select(keep_columns=[\"user_id\"])")
print(users.select(keep_columns=["user_id"]))
print()

users.select(keep_columns=["user_id"])
['user_id']
{'user_id': 0}
{'user_id': 1}
{'user_id': 2}
{'user_id': 3}
{'user_id': 4}
{'user_id': 5}
{'user_id': 6}
{'user_id': 7}
{'user_id': 8}
{'user_id': 9}
{'user_id': 10}



# SELECT user_id FROM users WHERE name = 'Dunn'

In [6]:
print('where(lambda row: row["name"] == "Dunn")')
print(users.where(lambda row: row["name"] == "Dunn")
               .select(keep_columns=["user_id"]))
print()

where(lambda row: row["name"] == "Dunn")
['user_id']
{'user_id': 1}



# SELECT LENGTH(name) AS name_length FROM users

In [9]:
def name_len(row): return len(row["name"])

print('with name_length:')
print(users.select(keep_columns=[],    #選擇條件函數的欄位  故挖空
                       additional_columns = { "name_length" : name_len }))
print()

with name_length:
['name_length']
{'name_length': 4}
{'name_length': 4}
{'name_length': 3}
{'name_length': 3}
{'name_length': 4}
{'name_length': 5}
{'name_length': 5}
{'name_length': 5}
{'name_length': 4}
{'name_length': 5}
{'name_length': 3}



# GROUP BY(合併)

In [8]:
def min_user_id(rows): return min(row["user_id"] for row in rows)

stats_by_length = users \
        .select(additional_columns={"name_len" : name_len}) \
        .group_by(group_by_columns=["name_len"],
                  aggregates={ "min_user_id" : min_user_id,
                               "num_users" : len })

print("stats by length")
print(stats_by_length)
print()

stats by length
['name_len', 'min_user_id', 'num_users']
{'min_user_id': 5, 'num_users': 4, 'name_len': 5}
{'min_user_id': 2, 'num_users': 3, 'name_len': 3}
{'min_user_id': 0, 'num_users': 4, 'name_len': 4}



In [14]:
def first_letter_of_name(row):
        return row["name"][0] if row["name"] else ""

def average_num_friends(rows):
        return sum(row["num_friends"] for row in rows) / len(rows)

def enough_friends(rows):
        return average_num_friends(rows) > 1

avg_friends_by_letter = users \
        .select(additional_columns={'first_letter' : first_letter_of_name}) \
        .group_by(group_by_columns=['first_letter'],
                  aggregates={ "avg_num_friends" : average_num_friends },
                  having=enough_friends)

print("avg friends by letter")
print(avg_friends_by_letter)
print()


avg friends by letter
['first_letter', 'avg_num_friends']
{'avg_num_friends': 2.5, 'first_letter': 'K'}
{'avg_num_friends': 3.0, 'first_letter': 'T'}
{'avg_num_friends': 2.5, 'first_letter': 'C'}
{'avg_num_friends': 1.5, 'first_letter': 'H'}
{'avg_num_friends': 2.0, 'first_letter': 'D'}
{'avg_num_friends': 3.0, 'first_letter': 'S'}



In [15]:
def sum_user_ids(rows): return sum(row["user_id"] for row in rows)

user_id_sum = users \
        .where(lambda row: row["user_id"] > 1) \
        .group_by(group_by_columns=[],
                  aggregates={ "user_id_sum" : sum_user_ids })

print("user id sum")
print(user_id_sum)
print()

user id sum
['user_id_sum']
{'user_id_sum': 54}



# ORDER BY

In [16]:
friendliest_letters = avg_friends_by_letter \
        .order_by(lambda row: -row["avg_num_friends"]) \
        .limit(4)

print("friendliest letters")
print(friendliest_letters)
print()

friendliest letters
['first_letter', 'avg_num_friends']
{'avg_num_friends': 3.0, 'first_letter': 'T'}
{'avg_num_friends': 3.0, 'first_letter': 'S'}
{'avg_num_friends': 2.5, 'first_letter': 'K'}
{'avg_num_friends': 2.5, 'first_letter': 'C'}



 # JOINs

In [17]:
user_interests = Table(["user_id", "interest"])
user_interests.insert([0, "SQL"])
user_interests.insert([0, "NoSQL"])
user_interests.insert([2, "SQL"])
user_interests.insert([2, "MySQL"])

sql_users = users \
    .join(user_interests,left_join=True) \
    

print("sql users")
print(sql_users)
print()

sql users
['user_id', 'name', 'num_friends', 'interest']
{'user_id': 0, 'num_friends': 0, 'name': 'Hero', 'interest': 'SQL'}
{'user_id': 0, 'num_friends': 0, 'name': 'Hero', 'interest': 'NoSQL'}
{'user_id': 1, 'num_friends': 2, 'name': 'Dunn', 'interest': None}
{'user_id': 2, 'num_friends': 3, 'name': 'Sue', 'interest': 'SQL'}
{'user_id': 2, 'num_friends': 3, 'name': 'Sue', 'interest': 'MySQL'}
{'user_id': 3, 'num_friends': 3, 'name': 'Chi', 'interest': None}
{'user_id': 4, 'num_friends': 3, 'name': 'Thor', 'interest': None}
{'user_id': 5, 'num_friends': 2, 'name': 'Clive', 'interest': None}
{'user_id': 6, 'num_friends': 3, 'name': 'Hicks', 'interest': None}
{'user_id': 7, 'num_friends': 2, 'name': 'Devin', 'interest': None}
{'user_id': 8, 'num_friends': 2, 'name': 'Kate', 'interest': None}
{'user_id': 9, 'num_friends': 3, 'name': 'Klein', 'interest': None}
{'user_id': 10, 'num_friends': 1, 'name': 'Jen', 'interest': None}



In [18]:
def count_interests(rows):
        """counts how many rows have non-None interests"""
        return len([row for row in rows if row["interest"] is not None])

user_interest_counts = users \
        .join(user_interests, left_join=True) \
        .group_by(group_by_columns=["user_id"],
                  aggregates={"num_interests" : count_interests })

print("user interest counts")
print(user_interest_counts)

user interest counts
['user_id', 'num_interests']
{'user_id': 10, 'num_interests': 0}
{'user_id': 5, 'num_interests': 0}
{'user_id': 0, 'num_interests': 2}
{'user_id': 6, 'num_interests': 0}
{'user_id': 1, 'num_interests': 0}
{'user_id': 7, 'num_interests': 0}
{'user_id': 2, 'num_interests': 2}
{'user_id': 8, 'num_interests': 0}
{'user_id': 3, 'num_interests': 0}
{'user_id': 9, 'num_interests': 0}
{'user_id': 4, 'num_interests': 0}
